In [30]:
def setup_cluster():
   from dask_drmaa import DRMAACluster
   import os
   
   cluster_kwargs_pass = {}    
   cluster_kwargs_pass.setdefault(
           'template',
           {
               'args': [
               '--nthreads', '1',
               '--local-directory', '/scratch/' + os.environ['USER']],
               'jobEnvironment': os.environ
           }
       )
   cluster = DRMAACluster(**cluster_kwargs_pass)
   return cluster

In [31]:
cluster = setup_cluster()
from dask.distributed import Client
client = Client(cluster)
cluster.start_workers(20)
client

Client Scheduler: tcp://10.36.107.16:46574 Dashboard: http://10.36.107.16:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [33]:
from fish.image.zds import ZDS 

In [34]:
dset = ZDS('/groups/ahrens/ahrenslab/davis/data/spim/raw/20180314/f1/5dpf_gfap_h2b_gcamp_x_huc_rgeco_x_ec43_opto_1_20180314_124124/')

In [35]:
import dask.array as da
from h5py import File

In [36]:
dars = da.stack([da.from_array(File(f , 'r')['default'], chunks=dset.shape[1:]) for f in dset.files])

In [37]:
dars

dask.array<stack, shape=(3515, 41, 1024, 2048), dtype=uint16, chunksize=(1, 41, 1024, 2048)>

In [40]:
client.restart()

Client Scheduler: tcp://10.36.107.16:46574 Dashboard: http://10.36.107.16:8787/status,Cluster Workers: 20 Cores: 20 Memory: 168.72 GB


In [41]:
client.compute(dars.mean((1,2,3)), sync=True)

/groups/ahrens/home/weiz/anaconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


array([103.44146784, 103.62579326, 104.14051879, ..., 102.98012865,
       102.98364861, 102.98867338])

In [42]:
nblocks = [16, 16]
transposed =dars.rechunk(chunks=(dars.shape[0], 1, dars.shape[2]//nblocks[0], dars.shape[3]//nblocks[1]))

In [ ]:
mean_im = transposed.mean(0).compute()

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

In [ ]:
client.close()
cluster.close()